<a href="https://colab.research.google.com/github/ahaque12/fiddler-break-bell-curve/blob/main/Fiddler_on_the_proof_Break_the_Bell_Curve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://thefiddler.substack.com/p/can-you-break-the-bell-curve

# Problem

Suppose you have a board like the one shown below. The board’s topmost row has three pins (and two slots for a ball to pass through), while the bottommost row has two pins (and three slots for a ball to pass through). The remaining rows alternate between having three pins and two pins.

But instead of the 12 rows of pins in the illustrative diagram, suppose the board has many, many rows. And at the very bottom of the board, just below the two bottommost pins, are three buckets, labeled A, B, and C from left to right.


Whenever a ball encounters one of the leftmost pins, it travels down the right side of it to the next row. And whenever a ball encounters one of the rightmost pins, it travels down the left side of it to the next row.

But this isn’t your garden variety bean machine. Whenever a ball encounters any of the other pins, it has a 75 percent chance of traveling down the right side of that pin, and a 25 percent chance of traveling down the left side of that pin.

A single ball is about to be dropped into the left slot at the top of the board. What is the probability that the ball ultimately lands in bucket A, the leftmost slot at the bottom?

# Approach
The transition matrix is
$$
\begin{align}
X &= \begin{bmatrix}
1 & 0 \\
.25 & .75 \\
0 & 1
      \end{bmatrix} \\
Y &= \begin{bmatrix}
              .25 & .75 & 0 \\
              0 & .25 & .75
            \end{bmatrix} \\
Z &= YX
\end{align}
$$
You need to proxy $Z^n$ as $n$ goes to $\infty$. Take the eigenvalues and eigenvectors of $Z$ and put the eigenvalues at the diagonal of a matrix $D$. Then take $D^\infty$. $Z_\infty$ is then $V D^\infty V^-1$. This is
$$
Z_\infty Y = \begin{bmatrix}
0.025 & 0.3 & 0.675 \\
0.025 & 0.3  & 0.675 \\
0.025 & 0.3  & 0.675
\end{bmatrix}
$$
which gives you a probability of .025.

In [1]:
import numpy as np

In [2]:
# rev
x = np.array([[1, 0],
              [.25, .75],
              [0, 1]])
y = np.array([
              [.25, .75, 0],
              [0, .25, .75]
            ])

x @ y

array([[0.25  , 0.75  , 0.    ],
       [0.0625, 0.375 , 0.5625],
       [0.    , 0.25  , 0.75  ]])

In [3]:
# Take y@x ^ inf
z = y @ x
eigenvalues, eigenvectors = np.linalg.eig(z)
eigenvalues, eigenvectors

(array([0.375, 1.   ]),
 array([[-0.99388373, -0.70710678],
        [ 0.11043153, -0.70710678]]))

In [4]:
D = np.diag(eigenvalues)
D

array([[0.375, 0.   ],
       [0.   , 1.   ]])

In [5]:
D_inf = np.where(np.abs(D) < 1, 0, D)  # Simplified for 0 and 1
D_inf = np.where(D == 1, 1, D_inf)      # Keep 1 as 1
D_inf

array([[0., 0.],
       [0., 1.]])

In [6]:
T = eigenvectors @ D_inf @ np.linalg.inv(eigenvectors)
T

array([[0.1, 0.9],
       [0.1, 0.9]])

In [7]:
S = np.array([1, 0]).reshape((1, 2))
S @ (T @ y)

array([[0.025, 0.3  , 0.675]])

In [8]:
T_b = np.linalg.matrix_power(z, 100)
S @ T_b @ y, T_b @ y

(array([[0.025, 0.3  , 0.675]]),
 array([[0.025, 0.3  , 0.675],
        [0.025, 0.3  , 0.675]]))

# Extra Credit

Suppose you have the board below, which starts with a row with six pins (and five slots), followed by a row with five pins (and six slots), and then back to six pins in an alternating pattern. Balls are only allowed to enter through the middle three slots on top.


Your goal is to create a trapezoidal distribution along one of the rows with six slots, which have been highlighted with dotted lines in the diagram above. More precisely, the frequency of balls passing through the six slots from left to right should be x−y, x, x+y, x+y, x, x−y, for some values of x and y with x ≥ y.

Suppose the ratio by which you drop balls into the top three middle slots, from left to right, is a : b : c, with a + b + c = 1. Find all ordered triples (a, b, c) that result in a trapezoidal distribution in at least one row with six slots.

# Approach
If you sum the trapezoidal distribution to 1 you find that $x$ is $\frac{1}{6}$. That implies the second and second to last slot must be $\frac{1}{6}$ at one step prior to converging. Therefore the initial distribution of three slots must be symmetric i.e., $a=c$. $b = \frac{2^n}{6}$ where $n \geq 1$. That leaves:

$$
\begin{align}
a, b, c &= \frac{1}{3}, \frac{1}{3}, \frac{1}{3} \\
&= \frac{1}{6},\frac{2}{3},\frac{1}{6}
\end{align}
$$

In [9]:
A = np.array([
    [.5, .5, 0, 0, 0, 0],
    [0, .5, .5, 0, 0, 0],
    [0, 0, .5, .5, 0, 0],
    [0, 0, 0, .5, .5, 0],
    [0, 0, 0, 0, .5, .5],
])

B = np.array([
    [1, 0, 0, 0, 0],
    [.5, .5, 0, 0, 0],
    [0, .5, .5, 0, 0],
    [0, 0, .5, .5, 0],
    [0, 0, 0, .5, .5],
    [0, 0, 0, 0, 1],
])

A.sum(axis=1), B.sum(axis=1)

(array([1., 1., 1., 1., 1.]), array([1., 1., 1., 1., 1., 1.]))

In [10]:
def trap_loss(x):
  x = x.flatten()
  y = (1/6) - x[0]
  return (x[2] - ((1/6) + y))**2

s = np.array([0, 1, 0, 0, 0]).reshape((1, 5))

print(trap_loss(np.array([1/6, 1/6, 1/6, 1/6, 1/6, 1/6])))
print(trap_loss(s @ A))

0.0
0.027777777777777783


In [11]:
def simulate(s, MAX=2**10):
  prior_state = np.zeros(6)
  state = s.copy()
  i = 0
  while True:
    out = state.shape[1] == 6
    if out:
      state = state @ B
    else:
      state = state @ A

    if not out and (trap_loss(state) < 1e-16):
      return True, state
    elif not out and np.array_equal(prior_state, state):
      print(i, "Converged")
      break
    elif not out:
      prior_state = state
      # print("{:.4f}, {}".format(trap_loss(state), state))
    elif i > MAX:
      break
    i += 1
  return False, state

print(s)
simulate(s)

[[0 1 0 0 0]]
696 Converged


(False, array([[0.1, 0.2, 0.2, 0.2, 0.2, 0.1]]))

In [12]:
simulate(np.array([0, 0, 1, 0, 0]).reshape((1, 5)))

176 Converged


(False, array([[0.1, 0.2, 0.2, 0.2, 0.2, 0.1]]))

In [13]:
simulate(np.array([0, 1/2, 0, 1/2, 0]).reshape((1, 5)))

170 Converged


(False, array([[0.1, 0.2, 0.2, 0.2, 0.2, 0.1]]))

In [14]:
# Valid distribution
simulate(np.array([0, 1/3, 1/3, 1/3, 0]).reshape((1, 5)))

(True,
 array([[0.        , 0.16666667, 0.33333333, 0.33333333, 0.16666667,
         0.        ]]))

In [15]:
# Valid distribution
simulate(np.array([0, 1/6, 2/3, 1/6, 0]).reshape((1, 5)))

(True,
 array([[0.046875  , 0.16666667, 0.28645833, 0.28645833, 0.16666667,
         0.046875  ]]))